!pip install psycopg2

In [14]:
import psycopg2
import pandas as pd
import APIkey

In [16]:
pwd = APIkey.progre_key["password"]

In [17]:
con = psycopg2.connect(database="mid_term_project", user="lhl_student", password=pwd, host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")

In [21]:
sql = 'select count(1) from flights'

### Facts about flights table
total records : 15927485 

total columns: 42

In [24]:
flight_count

,count
0,15927485


In [12]:
df_flight = pd.read_sql_query(sql,con) 

In [25]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fl_date              100 non-null    object 
 1   mkt_unique_carrier   100 non-null    object 
 2   branded_code_share   100 non-null    object 
 3   mkt_carrier          100 non-null    object 
 4   mkt_carrier_fl_num   100 non-null    int64  
 5   op_unique_carrier    100 non-null    object 
 6   tail_num             100 non-null    object 
 7   op_carrier_fl_num    100 non-null    int64  
 8   origin_airport_id    100 non-null    int64  
 9   origin               100 non-null    object 
 10  origin_city_name     100 non-null    object 
 11  dest_airport_id      100 non-null    int64  
 12  dest                 100 non-null    object 
 13  dest_city_name       100 non-null    object 
 14  crs_dep_time         100 non-null    int64  
 15  dep_time             100 non-null    floa

### Table **flights_test**

total number of records: 660556
total columns: 20
fl_date: 2020-1-1 to 2020-01-31
crs_elapsed_time mean : 144.95
distance mean: 795


In [27]:
sql = 'select * from flights_test limit 100'

In [28]:
df_test = pd.read_sql(sql,con)

In [29]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   fl_date             100 non-null    datetime64[ns]
 1   mkt_unique_carrier  100 non-null    object        
 2   branded_code_share  100 non-null    object        
 3   mkt_carrier         100 non-null    object        
 4   mkt_carrier_fl_num  100 non-null    int64         
 5   op_unique_carrier   100 non-null    object        
 6   tail_num            100 non-null    object        
 7   op_carrier_fl_num   100 non-null    int64         
 8   origin_airport_id   100 non-null    int64         
 9   origin              100 non-null    object        
 10  origin_city_name    100 non-null    object        
 11  dest_airport_id     100 non-null    int64         
 12  dest                100 non-null    object        
 13  dest_city_name      100 non-null    object        


In [30]:
df_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333
